In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import joblib
from tensorflow.keras.models import load_model
import os
from sklearn.preprocessing import MinMaxScaler

In [11]:

# List of stock tickers to train models on
tickers = ["AAPL", "GOOGL", "MSFT", "TSLA", "AMZN", "NFLX", "META"]

# Folder to store models and scalers
if not os.path.exists("models"):
    os.makedirs("models")

for ticker in tickers:
    model_path = f"models/{ticker}_lstm.h5"
    scaler_path = f"models/{ticker}_scaler.pkl"

    if os.path.exists(model_path) and os.path.exists(scaler_path):
        print(f"Model for {ticker} already exists. Skipping training...\n")
        continue  # Skip training if model already exists

    print(f"Training model for {ticker}...")

    # -------------------- Step 1: Download Data --------------------
    start_date = "2020-01-01"
    end_date = "2024-12-31"
    df = yf.download(ticker, start=start_date, end=end_date)

    # -------------------- Step 2: Preprocess Data --------------------
    scaler = MinMaxScaler(feature_range=(0, 1))
    df["Close_Scaled"] = scaler.fit_transform(df["Close"].values.reshape(-1, 1))

    sequence_length = 60  # Use past 60 days to predict next day
    X, y = [], []
    for i in range(len(df) - sequence_length):
        X.append(df["Close_Scaled"].iloc[i : i + sequence_length].values)
        y.append(df["Close_Scaled"].iloc[i + sequence_length])

    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))  # Reshape for LSTM

    # -------------------- Step 3: Build LSTM Model --------------------
    model = Sequential([
        LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)),
        Dropout(0.2),
        LSTM(units=50, return_sequences=False),
        Dropout(0.2),
        Dense(units=25),
        Dense(units=1)  # Predicting 1 value (next day's price)
    ])

    model.compile(optimizer="adam", loss="mean_squared_error")

    # -------------------- Step 4: Train Model --------------------
    model.fit(X, y, batch_size=32, epochs=20, verbose=1)

    # -------------------- Step 5: Save Model & Scaler --------------------
    model.save(model_path)  # Save model
    joblib.dump(scaler, scaler_path)  # Save scaler
    print(f"Model for {ticker} saved!\n")

Model for AAPL already exists. Skipping training...

Model for GOOGL already exists. Skipping training...

Model for MSFT already exists. Skipping training...

Model for TSLA already exists. Skipping training...

Model for AMZN already exists. Skipping training...

Model for NFLX already exists. Skipping training...

Model for META already exists. Skipping training...



In [20]:
def predict_future_prices(ticker, days):
    model_path = f"models/{ticker}_lstm.h5"
    scaler_path = f"models/{ticker}_scaler.pkl"
    
    if not os.path.exists(model_path) or not os.path.exists(scaler_path):
        print(f"Model or scaler missing for {ticker}. Exiting...")
        return None
    
    model = load_model(model_path)
    scaler = joblib.load(scaler_path)
    
    # Fetch latest stock data
    df = yf.download(ticker, period="70d")
    if df.shape[0] < 60:
        print(f"Not enough data to predict for {ticker}. Exiting...")
        return None
    
    last_60_days = df["Close"].values[-60:].reshape(-1, 1)
    last_60_days_scaled = scaler.transform(last_60_days)
    
    predictions = []
    input_seq = last_60_days_scaled.copy()
    
    for _ in range(days):
        X_test = np.array([input_seq])
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
        
        predicted_price_scaled = model.predict(X_test)
        predicted_price = scaler.inverse_transform(predicted_price_scaled)[0][0]
        predictions.append(predicted_price)
        
        # Update input sequence for next prediction
        input_seq = np.append(input_seq[1:], predicted_price_scaled, axis=0)
    
    return predictions

ticker = "AAPL"
days = 7
predicted_prices = predict_future_prices(ticker, days)
if predicted_prices:
    print(f"Predicted prices for {ticker} over next {days} days: {np.array(predicted_prices).astype(float)}")


[*********************100%***********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Predicted prices for AAPL over next 7 days: [239.1337738  239.2328186  238.99214172 238.53462219 237.93995667
 237.25895691 236.52372742]
